In [1]:
import keras
keras.__version__
import matplotlib.pyplot as plt
from keras import regularizers
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import tensorflow as tf

In [2]:
# #GPU显存
# import tensorflow as tf
# import numpy as np
# config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
# tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
# #os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
#导入数据
import os
original_dataset_dir="/home/featurize/data/train_val/"
train_dir=os.path.join(original_dataset_dir,'train')
validation_dir=os.path.join(original_dataset_dir,'val')
#test_dir=os.path.join(original_dataset_dir,'test')


Let's put this in practice by using the convolutional base of the VGG16 network, trained on ImageNet, to extract interesting features from 
our cat and dog images, and then training a cat vs. dog classifier on top of these features.

The VGG16 model, among others, comes pre-packaged with Keras. You can import it from the `keras.applications` module. Here's the list of 
image classification models (all pre-trained on the ImageNet dataset) that are available as part of `keras.applications`:

* Xception
* InceptionV3
* ResNet50
* VGG16
* VGG19
* MobileNet

Let's instantiate the VGG16 model:

In [4]:
from tensorflow.keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224,3))

2022-04-19 10:55:27.577671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 10:55:27.586258: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 10:55:27.586566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 10:55:27.588023: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model = tf.keras.Sequential([
    conv_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256,kernel_regularizer=regularizers.l2(0.1),activation = 'relu'),
    #tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dense(13,activation = 'softmax')
])


This is what our model looks like now:

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 13)                3341      
                                                                 
Total params: 26,450,509
Trainable params: 26,450,509
Non-trainable params: 0
_________________________________________________________________


In [8]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 36


In [9]:
# conv_base.trainable = False

In [10]:
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 36


In [11]:
# conv_base.trainable = True

# set_trainable = False
# for layer in conv_base.layers:
#     if layer.name == 'block5_conv1':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [12]:
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,)
#       rotation_range=90,#图片翻转角度
#       width_shift_range=0.2,#图片随机水平偏移的幅度
#       height_shift_range=0.2,#图片随机垂直偏移的幅度
#       shear_range=0.2,#剪切强度
#       zoom_range=0.2,#随机放大
#       horizontal_flip=True,
#       fill_mode='nearest')
train_datagen = ImageDataGenerator(rescale = 1./255)
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(224, 224),
        batch_size=32,#有可能batch_size过大
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

model.compile(loss='categorical_crossentropy',#binary_crossentropy',#categorical_crossentropy
              optimizer=optimizers.RMSprop(learning_rate=1e-5),
              metrics=['acc'])




Found 987 images belonging to 13 classes.
Found 252 images belonging to 13 classes.


In [13]:
# callbacks = [
#     keras.callbacks.TensorBoard(
#     log_dir='car_part_recognition',
#     histogram_freq=1,
#     embeddings_freq=1)
# ]

In [14]:
train_generator.class_indices# 输出输入哪一类


{'Strawberry___Leaf_scorch': 0,
 'Strawberry___healthy': 1,
 '冻害': 2,
 '叶斑病': 3,
 '斜纹夜蛾': 4,
 '根腐病': 5,
 '灰霉病': 6,
 '病毒病': 7,
 '白粉虱': 8,
 '着色不良': 9,
 '缺素症': 10,
 '革腐病': 11,
 '黄萎病': 12}

In [15]:
history = model.fit(
      train_generator,
      epochs=50,
      validation_data=validation_generator,
      verbose=1)
      #callbacks=callbacks)


Epoch 1/50


2022-04-19 10:55:34.266193: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-04-19 10:55:38.077892: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


31/31 [==============================] - 32s 761ms/step - loss: 51.5463 - acc: 0.2320 - val_loss: 49.8360 - val_acc: 0.4643
Epoch 2/50
31/31 [==============================] - 19s 601ms/step - loss: 48.7618 - acc: 0.4873 - val_loss: 47.6507 - val_acc: 0.5357
Epoch 3/50
31/31 [==============================] - 19s 611ms/step - loss: 46.7331 - acc: 0.6130 - val_loss: 45.9407 - val_acc: 0.5992
Epoch 4/50
31/31 [==============================] - 19s 622ms/step - loss: 45.0168 - acc: 0.6879 - val_loss: 44.4275 - val_acc: 0.6270
Epoch 5/50
31/31 [==============================] - 19s 609ms/step - loss: 43.4108 - acc: 0.7639 - val_loss: 42.9785 - val_acc: 0.6825
Epoch 6/50
31/31 [==============================] - 20s 650ms/step - loss: 41.8633 - acc: 0.8186 - val_loss: 41.6645 - val_acc: 0.6865
Epoch 7/50
31/31 [==============================] - 19s 608ms/step - loss: 40.4417 - acc: 0.8693 - val_loss: 40.4966 - val_acc: 0.6548
Epoch 8/50
31/31 [==============================] - 18s 594ms/step

In [16]:
import pandas as pd
data = pd.DataFrame(history.history)

In [17]:
data

,loss,acc,val_loss,val_acc
0,51.546280,0.232016,49.836044,0.464286
1,48.761848,0.487335,47.650661,0.535714
2,46.733074,0.612969,45.940712,0.599206
3,45.016758,0.687943,44.427456,0.626984
4,43.410816,0.763931,42.978451,0.682540
5,41.863255,0.818642,41.664474,0.686508
6,40.441685,0.869301,40.496597,0.654762
7,39.041504,0.898683,39.001434,0.690476
8,37.731632,0.918946,37.798565,0.706349
9,36.356770,0.955420,36.609138,0.690476


In [18]:
data.to_csv('./实验记录/VGG19迁移学习.csv')

In [19]:
# model.save('草莓VGG19（224）.h5')

Let's plot our results again:

In [20]:
# model.save('car_part_ABCD(裁剪后).h5')